In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("winemag-data-130k-v2.csv")

In [3]:
df.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [4]:
df.shape

(129971, 14)

In [5]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

**Filtro por rango IQR:**

In [6]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
print ("1° Cuartil: ", Q1)
print ("3° Cuartil: ", Q3)
print ("Rango intercuartílico: ", IQR)

1° Cuartil:  17.0
3° Cuartil:  42.0
Rango intercuartílico:  25.0


In [7]:
df = df[(df['price'] < (1.5 * IQR + Q3)) & (df['price'] > (1.5 * IQR - Q1))]

**Analizo todas las entradas únicas (países):**

In [8]:
df['country'].unique()

array(['US', 'France', 'Germany', 'Argentina', 'Spain', 'Italy', 'Chile',
       'Australia', 'South Africa', 'Portugal', 'Israel', 'Hungary',
       'Austria', 'Canada', 'New Zealand', nan, 'Greece',
       'Czech Republic', 'Romania', 'Croatia', 'Uruguay', 'England',
       'Lebanon', 'Serbia', 'Brazil', 'Mexico', 'Slovenia', 'Turkey',
       'Moldova', 'Bulgaria', 'Georgia', 'Morocco', 'Switzerland', 'Peru',
       'Cyprus', 'Luxembourg'], dtype=object)

**Ahora establezco la transformación para clasificar los países en los continentes correspondientes, y lo almaceno en el nuevo DF:**

In [9]:
continents = {0:('Georgia', 'China', 'India'),
              1:('South Africa', 'Morocco', 'Egypt'),
              2:('US', 'Mexico', 'Canada'),
              3:('Argentina', 'Chile', 'Uruguay', 'Brazil', 'Peru'),
              5:('Italy', 'Portugal', 'Spain', 'France', 'Germany', 'Austria', 
                 'Israel', 'Hungary', 'Greece', 'Romania', 'Turkey', 'Czech Republic', 
                 'Slovenia', 'Luxembourg', 'Croatia', 'England', 'Lebanon', 'Serbia', 
                 'Moldova', 'Bulgaria', 'Cyprus', 'Armenia', 'Switzerland',
                 'Bosnia and Herzegovina', 'Ukraine', 'Slovakia', 'Macedonia'),
              6:('Australia', 'New Zealand')}

In [10]:
def get_continent(country):
    for ind, v in continents.items():
        if country in v:
            return ind
    return -1

In [11]:
df_filtered_aux = pd.DataFrame()

In [12]:
df_filtered_aux['continents'] = df['country'].apply(lambda x: get_continent(x))

**Ahora separo las columnas para el procesamiento is_sweet:**

La columna 'description' es necesaria, ya que aquí se buscará la ocurrencia de la palabra "sweet"

Ahora asigno 1 a las instancias donde ocurre la palabra "sweet", y 0 en caso contrario:

1 = "sweet";
0 != "sweet"

In [13]:
df_filtered_aux['is_sweet'] = df['description'].str.contains('sweet', na=-1, regex=True)

**A partir de ahora implemento price_4levels:**

Solo es importante la columna 'price'

"LOW" (precio menor o igual a $15);
"INTERMEDIATE" (precio entre $15 y $40);
"HIGH" (precio entre $40 y $70);
"VERY HIGH" (precio mayor a $70).

In [14]:
df_filtered = df['price']

In [15]:
def price_levels(v):
    if (v <= 15):
        return('LOW')
    elif ((v > 15) & (v <= 40)):
        return('INTERMEDIATE')
    elif ((v > 40) & (v <= 70)):
        return('HIGH')
    elif (v <= 15):
        return('VERY HIGH')
    else:
        return(-1)

In [16]:
df_filtered_aux['price_4levels'] = df['price'].apply(lambda x: price_levels(x))

**Ahora trabajo para obtener was_master_taste:**

"Roger Voss", "Michael Schachner", "Paul Gregutt", "Virginie Boone" son los maestros.

Coloco 1 si el vino fue analizado por uno de ellos y 0 en caso contrario

In [17]:
masters = ['Roger Voss', 'Michael Schachner', 'Paul Gregutt', 'Virginie Boone']

In [18]:
def is_masters(name):
    if name in masters:
        return 1
    else:
        return 0

In [19]:
df_filtered_aux['was_master_taster'] = df['taster_name'].apply(lambda x: is_masters(x))

**century_title:**

Asumiendo que el título del vino frecuentemente contiene el año del mismo, se debe proveer una columna booleana indicando si fue producido el siglo pasado (1900-1999) o el actual (2000-2019). Quizás puede servir la siguiente función de Pandas mediante esta regex: r'.*(\d{4}).*'.

Es necesaria la columna 'title'

In [37]:
def is_century(anio):
    if (anio >= 1900) & (anio <= 1999):
        return 0
    elif (anio >= 2000) & (anio <= 2019):
        return 1
    return -1

In [38]:
df_filtered_aux['century_title'] = df['title'].str.extract('.(\d{4}).').fillna(-1).astype(int)

In [40]:
df_filtered_aux['century_title'] = df_filtered_aux['century_title'].apply(lambda x: is_century(x))

**variety_ohe:**

Indicar con One Hot Encoding (OHE) mediante 3 columnas si el vino pertenece a alguna de las siguientes 3 categorías:
Red: Si entra en el top 8 de principales vinos rojos descriptos aquí.
White: Si entra en el top 7 de principales vinos blancos descriptos aquí.
Other: El resto de las categorías no cubiertas

0 = Red; 1 = White; 2 = Other

In [43]:
varieties = {0:('Syrah', 'Shiraz', 'Merlot', 'Cabernet Sauvignon',
              'Malbec', 'Pinot Noir', 'Zinfandel', 'Sangiovese',
              'Barbera'),
           1:('Chardonnay', 'Sauvignon Blanc', 'Semillon', 'Moscato',
              'Pinot Grigio', 'Gewürztraminer', 'Riesling')}

In [44]:
def which_variety(variety):
    for ind,v in varieties.items():
        if variety in v:
            return ind
    return 2

In [45]:
df_filtered_aux['variety_ohe'] = df['variety'].apply(lambda x: which_variety(x))

Ahora uso el equivalente OHE (OneHotEncoder) de Pandas: la función 'get_dummies'

multicol1 = pd.MultiIndex.from_tuples([('weight', 'kg'),
...                                        ('weight', 'pounds')])

In [50]:
pd.MultiIndex.from_tuples([('variety_ohe', 'a'),
                           ('variety_ohe', 'b'),
                           ('variety_ohe', 'c')])

MultiIndex(levels=[['variety_ohe'], ['a', 'b', 'c']],
           labels=[[0, 0, 0], [0, 1, 2]])

In [48]:
df_filtered_aux['variety_ohe'] = pd.get_dummies(df_filtered_aux['variety_ohe'])

In [49]:
df_filtered_aux

,continents,is_sweet,price_4levels,was_master_taster,century_title,variety_ohe
4,2,False,HIGH,1,1,1
7,5,False,INTERMEDIATE,1,1,0
9,5,False,INTERMEDIATE,1,1,0
11,5,False,INTERMEDIATE,1,1,0
12,2,False,INTERMEDIATE,1,1,1
15,5,False,INTERMEDIATE,0,1,0
16,3,False,INTERMEDIATE,1,1,1
18,5,False,INTERMEDIATE,1,1,0
19,2,False,INTERMEDIATE,0,1,0
20,2,False,INTERMEDIATE,0,1,0
